In [3]:
import glob
import os
import re

import pandas as pd
import spacy
from lxml import etree
from lxml.etree import XMLSyntaxError
from spacy import Language
from spacy_langdetect import LanguageDetector


@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()


NLP = spacy.load('de_core_news_sm')
NLP.add_pipe('language_detector')

ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [4]:
NAMESPACE: dict = {'tei': 'http://www.tei-c.org/ns/1.0'}
SAVINGS_PATH: str = '../data/plain-text/'

if not os.path.exists(SAVINGS_PATH):
    os.makedirs(SAVINGS_PATH)

In [ ]:
def get_letter_text(path: str ='../data/xml/*.xml') -> None:
    """Function for extracting and cleaning text from the XML/TEI letter files.

    :param path: File path given as string.
    :return: None.
    """
    for file in glob.glob(pathname=path)[:]:
        try:
            tree = etree.parse(file)

            letter_output: str = ""

            for text in tree.xpath('//tei:text//tei:body//text()', namespaces=NAMESPACE):
                text = text.lower()
                # Remove any expressions of type "[HERE NUMBER]" from the text.
                if re.match(r'.*\[.*].*', text):
                    letter_output += re.sub(r'\[.*]', '', text) + ' '
                # Remove the given statements from the text as they are not needed for performing SA.
                elif 'beyliegendes von' or 'paginierung des editors' in text:
                    letter_output += re.sub(r'beyliegendes von|paginierung des editors', '', text) + ' '
                # Remove any dates (except for dates in words, e.g. "14. Februar 1833").
                else:
                    letter_output += re.sub(r'^(0[1-9]|[12][0-9]|3[01])[.](0[1-9]|1[012])[.](19|20)[0-9]{2}', ' ',
                                            text) + ' '

            # Remove unnecessary whitespace and any further characters that are neither alphanumerical nor (singular) whitespace.
            letter_output = re.sub('\s{2,}', ' ', letter_output.lstrip())
            letter_output = re.sub('[^\w\s\d]|_', '', letter_output)

            """Using spaCy's available pipe for language detection to store only German letters. I also tried out to do this before preprocessing the texts but this lead to some major issues.
            """
            doc = NLP(letter_output)
            if doc._.language['language'] == 'de':
                with open(SAVINGS_PATH + re.sub(r'\.xml', '.txt', os.path.basename(file)), 'w') as f_tbw:
                    f_tbw.write(letter_output)

        except XMLSyntaxError:
            pass



In [ ]:
get_letter_text()

In [5]:
from textblob_de import TextBlobDE
from textblob_de import sentiments

In [ ]:
sentiment_dict: dict = {}

for file in glob.glob('../data/plain-text/*.txt')[:50]:
    with open(file, 'r') as f:
        text = f.read()
        letter_blob = TextBlobDE(text)
        #letter_blob = TextBlobDE(' '.join(letter_blob.words.lemmatize()))
        #print(letter_blob.words.lemmatize())
        sentiment_dict[os.path.basename(file)] = letter_blob.sentiment


s_df = pd.DataFrame.from_dict(sentiment_dict).transpose()
s_df.index.names = ['filename']
s_df.rename(columns={0: 'polarity', 1: 'subjectivity'}, inplace=True)
print(s_df)
#s_df.to_csv('../data/retrieved/sentiments.csv')


['genf', 'd', '1811', 'so', 'eben', 'erhalte', 'ich', 'ew', 'wohlgeb', 'v', 'j', 'wegen', 'eines', 'bücherpackets', 'die', 'exemplare', 'vom', '3ten', 'b', 'meiner', 'vorlesungen', 'und', 'andre', 'sachen', 'enthaltend', 'das', 'packet', 'ist', 'noch', 'nicht', 'in', 'die', 'stadt', 'hereingebracht', 'u', 'ich', 'bin', 'also', 'von', 'dessen', 'inhalt', 'nicht', 'genau', 'unterrichtet', 'auf', 'jeden', 'fall', 'ist', 'es', 'aber', 'richtig', 'und', 'wohlbehalten', 'eingetroffen', 'den', 'brief', 'vom', '6sten', 'dec', 'habe', 'ich', 'ebenfalls', 'schon', 'vor', 'einiger', 'zeit', 'empfangen', 'und', 'danke', 'ew', 'w', 'für', 'die', 'anweisung', 'der', 'summe', 'von', '225', 'rth', 'an', 'h', 'hofsecretär', 'ernst', 'in', 'dresden', 'gestern', 'ist', 'durch', 'den', 'sogenannten', 'fourgon', 'von', 'hier', 'aus', 'durch', 'die', 'schweiz', 'sorgfältig', 'in', 'wachsleinwand', 'eingepackt', 'der', '2te', 'band', 'meiner', 'gedichte', 'abgegangen', 'ich', 'werde', 'sogleich', 'die', 'mat